# 상가 점포명 모음 

In [1]:
!pwd

/content


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv('/content/drive/MyDrive/출발 드림팀/crawling/진형/df5.csv')

In [6]:
df.head()

,Unnamed: 0,상호명,행정동명,행정동코드
0,15000,금가옛날찜갈비,구로4동,1153055000
1,15001,감자탕생고기,구로4동,1153055000
2,15002,한우마당,구로4동,1153055000
3,15003,즉석우동·짜장,구로4동,1153055000
4,15004,정이호프,구로4동,1153055000


In [7]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 7.0 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


# 크롤링

In [8]:
pip install webdriver-manager

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
from selenium import webdriver
driver = webdriver.Chrome('/content/drive/MyDrive/출발 드림팀/kimwon/NaverMap/chromedriver')

<ipython-input-10-48c140329c89>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('/content/drive/MyDrive/출발 드림팀/kimwon/NaverMap/chromedriver')


AttributeError: ignored

In [120]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time


chromedriver = '/content/drive/MyDrive/출발 드림팀/kimwon/NaverMap/chromedriver' 
driver = webdriver.Chrome(chromedriver)


# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['행정동명'] + "%20" + df['상호명']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''


# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(2)
        df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR,"#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element(By.CSS_SELECTOR, "#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()


# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']


# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

<ipython-input-120-32629fa0e3a4>:13: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chromedriver)


AttributeError: ignored

In [ ]:
df

,상호명,행정동명,행정동코드,법정동코드,naver_keyword,naver_map_url
0,비지트,방배4동,1165062100,1165010100,방배4동%20비지트,https://m.place.naver.com/restaurant/1848424123
3,고향생막걸리,압구정동,1168054500,1168010700,압구정동%20고향생막걸리,https://m.place.naver.com/restaurant/1496470647
4,도전최강달인왕만두,고덕2동,1174056000,1174010200,고덕2동%20도전최강달인왕만두,https://m.place.naver.com/restaurant/1187890786
5,L.A.D,서교동,1144066000,1144012000,서교동%20L.A.D,https://m.place.naver.com/restaurant/1857845585
6,양푼이김치찌개,방이2동,1171056200,1171011100,방이2동%20양푼이김치찌개,https://m.place.naver.com/restaurant/1403941863
7,은진이네,공덕동,1144056500,1144010100,공덕동%20은진이네,https://m.place.naver.com/restaurant/1385321165
8,흥부보쌈,목1동,1147051000,1147010200,목1동%20흥부보쌈,https://m.place.naver.com/restaurant/1379617037
22,"수제비,잔치",신원동,1162065500,1162010200,"신원동%20수제비,잔치",https://m.place.naver.com/restaurant/34355311
28,본죽,청량리동,1123070500,1123010700,청량리동%20본죽,https://m.place.naver.com/restaurant/36561502
29,봉구비어삼양동사거리점,송천동,1130555500,1130510100,송천동%20봉구비어삼양동사거리점,https://m.place.naver.com/restaurant/36782171


In [ ]:
# 각 데이터들을 미리 리스트에 담은 다음, 마지막에 데이터 프레임에 합칠 것입니다.


chromedriver = 'chromedriver' 


vistor_review_qty_list = []
blog_review_qty_list = []
star_review_stars_list = []


driver = webdriver.Chrome(chromedriver) 


for i, url in enumerate(df['naver_map_url']):

    driver.get(url)
    time.sleep(2)
    # 네이버 지도의 유형 분류

    try:


        # 네이버 지도의 유형 분류
        vistor_review_qty = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[2]/a/em').text

        # 블로그 리뷰 수
        blog_review_qty = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[3]/a/em').text
        # 블로그 별점 점수
        star_review_stars = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/span[1]/em').text

        vistor_review_qty_list.append(vistor_review_qty)
        blog_review_qty_list.append(blog_review_qty)
        star_review_stars_list.append(star_review_stars)

    # 리뷰가 없는 업체는 크롤링에 오류가 뜨므로 표기해둡니다.
    except Exception as e1:
        print(f"{i}행 문제가 발생")
        
        # 리뷰가 없으므로 null을 임시로 넣어줍니다.
        vistor_review_qty_list.append('null')
        blog_review_qty_list.append('null')
        star_review_stars_list.append('null')


        
driver.quit()

df['naver_visitor_review_qty'] = vistor_review_qty_list  # 네이버 상세페이지에서 크롤링한 업체 유형
df['naver_star_point'] = star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
df['naver_blog_review_qty'] = blog_review_qty_list  # 네이버 상세페이지에 나온 블로그 리뷰의 총 개수

170 117 4.82
54 7 4.23
  
  
240 283 4.62
5행 문제가 발생
  
314 23 4.27
  
  
113 6 4.49


/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_16932/3387214688.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_visitor_review_qty'] = vistor_review_qty_list  # 네이버 상세페이지에서 크롤링한 업체 유형
/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_16932/3387214688.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['naver_star_point'] = star_review_stars_list  # 네이버 상세페이지에서 평가한 별점 평점
/var/folders/s3/xdg3f00d5gz2dnfsw7wsjyjc0000gn/T/ipykernel_16932/3387214688.py:52: SettingWithCopyW

# 여기까지하고 저장하쇼!!! 

In [ ]:
df.to_csv('~~~ ')

,상호명,행정동명,행정동코드,법정동코드,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty,naver_visitor_review_qty
0,비지트,방배4동,1165062100,1165010100,방배4동%20비지트,https://m.place.naver.com/restaurant/1848424123,null,4.82,null,null,117,170
3,고향생막걸리,압구정동,1168054500,1168010700,압구정동%20고향생막걸리,https://m.place.naver.com/restaurant/1496470647,null,4.23,null,null,7,54
4,도전최강달인왕만두,고덕2동,1174056000,1174010200,고덕2동%20도전최강달인왕만두,https://m.place.naver.com/restaurant/1187890786,null,,null,null,,
5,L.A.D,서교동,1144066000,1144012000,서교동%20L.A.D,https://m.place.naver.com/restaurant/1857845585,null,,null,null,,
6,양푼이김치찌개,방이2동,1171056200,1171011100,방이2동%20양푼이김치찌개,https://m.place.naver.com/restaurant/1403941863,null,4.62,null,null,283,240
7,은진이네,공덕동,1144056500,1144010100,공덕동%20은진이네,https://m.place.naver.com/restaurant/1385321165,null,null,null,null,null,null
8,흥부보쌈,목1동,1147051000,1147010200,목1동%20흥부보쌈,https://m.place.naver.com/restaurant/1379617037,null,,null,null,,
22,"수제비,잔치",신원동,1162065500,1162010200,"신원동%20수제비,잔치",https://m.place.naver.com/restaurant/34355311,null,4.27,null,null,23,314
28,본죽,청량리동,1123070500,1123010700,청량리동%20본죽,https://m.place.naver.com/restaurant/36561502,null,,null,null,,
29,봉구비어삼양동사거리점,송천동,1130555500,1130510100,송천동%20봉구비어삼양동사거리점,https://m.place.naver.com/restaurant/36782171,null,,null,null,,


In [ ]:

# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
